# **SCHOOL BUDDY DATA ANALYSIS**

# **Problem Statement**

School Buddy is a new startup that helps students in identifying best schools as per their stream preferences. There are three streams in which students seek admission after passing their 10th board exams. Subjects for each of these streams are as follows:

Arts - Hindi, English, History, Geography and Civics

Science - Mathematics, Physics, Chemistry, Biology and Computer Science

Commerce - Hindi, English, Mathematics, Computer Science and Physical Education

School Buddy has collected marks scored by the same students for all the subjects for 3 consecutive years.



In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns #visualisation
import matplotlib.pyplot as plt #visualisation
%matplotlib inline
sns.set(color_codes=True)
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

# Datasets

In [ ]:
data2019 = pd.read_excel('/content/Bangalore Schools 2019.xlsx', sheet_name=None)

In [ ]:
data2020 = pd.read_excel('/content/Bangalore Schools 2020.xlsx', sheet_name=None)

In [ ]:
data2021 = pd.read_excel('/content/Bangalore Schools 2021.xlsx', sheet_name=None)

In [ ]:

data2019['Vidya Mandir'].head() #we can extract data based on excel workbook and specific sheet details


,Student Roll,Student Name,Hindi,English,Mathematics,Physics,Chemistry,Biology,History,Geography,Civics,Computer Science,Physical Education
0,1001,Akash Ahuja,30,95,78,92,84,36,47,82,44,14,74
1,1002,Mohit Srivatsav,32,46,65,53,28,77,36,76,74,77,27
2,1003,Anshika Mehra,87,91,74,81,88,100,45,37,10,44,11
3,1004,Visakha Tamboli,65,83,83,78,45,61,16,89,52,23,50
4,1005,Raju Deshmukh,84,66,98,62,68,99,70,61,65,75,87


In [ ]:
excel_files = {
    '2019': pd.ExcelFile('/content/Bangalore Schools 2019.xlsx'),
    '2020': pd.ExcelFile('/content/Bangalore Schools 2020.xlsx'),
    '2021': pd.ExcelFile('/content/Bangalore Schools 2021.xlsx')
}


In [ ]:
sheet_names = {year: excel_files[year].sheet_names for year in excel_files}

print(sheet_names)

{'2019': ['Vidya Mandir', 'St. Joseph', 'DPS', 'Birla HS', 'International'], '2020': ['Vidya Mandir', 'St. Joseph', 'DPS', 'Birla HS', 'International'], '2021': ['Vidya Mandir', 'St. Joseph', 'DPS', 'Birla HS', 'International']}


Question 1: Reward the top performer (student) of each school based on cumulative marks scored in last three years for all the subjects.

In [ ]:
def load_and_combine_data(school_name):
    combined_data = []
    for year in excel_files:
        df = excel_files[year].parse(school_name)
        df['Year'] = year
        combined_data.append(df)
    combined_df = pd.concat(combined_data)
    return combined_df

In [ ]:
school_name = 'Vidya Mandir'
data = load_and_combine_data(school_name)
data.head()


,Student Roll,Student Name,Hindi,English,Mathematics,Physics,Chemistry,Biology,History,Geography,Civics,Computer Science,Physical Education,Year
0,1001,Akash Ahuja,30,95,78,92,84,36,47,82,44,14,74,2019
1,1002,Mohit Srivatsav,32,46,65,53,28,77,36,76,74,77,27,2019
2,1003,Anshika Mehra,87,91,74,81,88,100,45,37,10,44,11,2019
3,1004,Visakha Tamboli,65,83,83,78,45,61,16,89,52,23,50,2019
4,1005,Raju Deshmukh,84,66,98,62,68,99,70,61,65,75,87,2019


In [ ]:
def calculate_cumulative_marks(df, subject_columns):
    # Calculate total marks for each student by summing across subject columns
    df['Cumulative_Marks'] = df.groupby('Student Roll')[subject_columns].transform('sum').sum(axis=1)
    return df

In [ ]:

subject_columns = ['Hindi', 'English', 'Mathematics', 'Physics', 'Chemistry', 'Biology', 'Computer Science', 'Geography', 'History', 'Civics', 'Physical Education']
data = calculate_cumulative_marks(data, subject_columns)

data.head()

,Student Roll,Student Name,Hindi,English,Mathematics,Physics,Chemistry,Biology,History,Geography,Civics,Computer Science,Physical Education,Year,Cumulative_Marks
0,1001,Akash Ahuja,30,95,78,92,84,36,47,82,44,14,74,2019,2119
1,1002,Mohit Srivatsav,32,46,65,53,28,77,36,76,74,77,27,2019,1951
2,1003,Anshika Mehra,87,91,74,81,88,100,45,37,10,44,11,2019,2052
3,1004,Visakha Tamboli,65,83,83,78,45,61,16,89,52,23,50,2019,1758
4,1005,Raju Deshmukh,84,66,98,62,68,99,70,61,65,75,87,2019,2298


In [ ]:
def find_top_performer(df):
    top_performer = df.loc[df.groupby('Student Roll')['Cumulative_Marks'].idxmax()]
    return top_performer.sort_values(by='Cumulative_Marks', ascending=False).iloc[0]

top_performer = find_top_performer(data)
print(top_performer)

Student Roll                     5001
Student Name          Swetashi Aiyyar
Hindi                              86
English                            64
Mathematics                        77
Physics                            69
Chemistry                          70
Biology                            64
History                            20
Geography                          59
Civics                             73
Computer Science                   53
Physical Education                 64
Year                             2019
Cumulative_Marks                 2166
Name: 0, dtype: object


In [ ]:

top_performers = []
for year in excel_files:
    for school_name in sheet_names[year]:
        data = load_and_combine_data(school_name)
        # Pass the list of subject columns to the function
        data = calculate_cumulative_marks(data, ['Hindi', 'English', 'Mathematics', 'Physics', 'Chemistry', 'Biology', 'Computer Science', 'Geography', 'History', 'Civics', 'Physical Education'])
        top_performer = find_top_performer(data)
        top_performers.append({
            'School': school_name,
            'Top_Performer': top_performer['Student Name'], # Use 'Student Roll' here as well
            'Cumulative_Marks': top_performer['Cumulative_Marks']
        })

top_performers_df = pd.DataFrame(top_performers)
print(top_performers_df)

           School    Top_Performer  Cumulative_Marks
0    Vidya Mandir     Nisha Saxena              2320
1      St. Joseph   Agriya Marandi              2056
2             DPS        Jivan Rao              2043
3        Birla HS   Hashmukh Patel              2209
4   International  Swetashi Aiyyar              2166
5    Vidya Mandir     Nisha Saxena              2320
6      St. Joseph   Agriya Marandi              2056
7             DPS        Jivan Rao              2043
8        Birla HS   Hashmukh Patel              2209
9   International  Swetashi Aiyyar              2166
10   Vidya Mandir     Nisha Saxena              2320
11     St. Joseph   Agriya Marandi              2056
12            DPS        Jivan Rao              2043
13       Birla HS   Hashmukh Patel              2209
14  International  Swetashi Aiyyar              2166


**Question 2 : Rank each student within their own school based on their total marks scored in the year 2020 and compare the marks of Rank 10 for each school by arranging them in descending order**


In [ ]:
excel_files = {
    '2019': pd.ExcelFile('/content/Bangalore Schools 2019.xlsx'),
    '2020': pd.ExcelFile('/content/Bangalore Schools 2020.xlsx'),
    '2021': pd.ExcelFile('/content/Bangalore Schools 2021.xlsx')
}


In [ ]:
sheet_names = excel_files['2020'].sheet_names

In [ ]:
def load_data_for_year(school_name, year):
    df = excel_files[year].parse(school_name)
    return df

In [ ]:
subject_columns = {
    'Arts': ['Hindi', 'English', 'History', 'Geography', 'Civics'],
    'Science': ['Mathematics', 'Physics', 'Chemistry', 'Biology', 'Computer Science'],
    'Commerce': ['Hindi', 'English', 'Mathematics', 'Computer Science', 'Physical Education']
}

In [ ]:
all_subject_columns = list(set([subject for subjects in subject_columns.values() for subject in subjects]))

In [ ]:
def calculate_total_marks(df, subject_columns):
    df['Total_Marks'] = df[subject_columns].sum(axis=1)
    return df

In [ ]:
def rank_students(df):
    df['Rank'] = df['Total_Marks'].rank(ascending=False, method='min')
    return df

In [ ]:
ranked_students = []
for school_name in sheet_names:
    # Load data for the school for the year 2020
    data = load_data_for_year(school_name, '2020')

    # Calculate total marks
    data = calculate_total_marks(data, all_subject_columns)

    # Rank students within the school
    ranked_data = rank_students(data)

    # Add school name to the DataFrame
    ranked_data['School'] = school_name

    # Append the ranked data to the list
    ranked_students.append(ranked_data)

In [ ]:
# Concatenate all the ranked data into a single DataFrame
ranked_students_df = pd.concat(ranked_students)

# Display the result
ranked_students_df.head()

,Student Roll,Student Name,Hindi,English,Mathematics,Physics,Chemistry,Biology,History,Geography,Civics,Computer Science,Physical Education,Total_Marks,Rank,School
0,1001,Akash Ahuja,39,74,87,92,84,36,47,82,44,81,78,744,3.0,Vidya Mandir
1,1002,Mohit Srivatsav,72,38,97,53,28,77,36,76,74,68,81,700,11.0,Vidya Mandir
2,1003,Anshika Mehra,82,52,94,81,88,100,45,37,10,36,69,694,12.0,Vidya Mandir
3,1004,Visakha Tamboli,44,68,62,78,45,61,16,89,52,37,45,597,18.0,Vidya Mandir
4,1005,Raju Deshmukh,83,49,44,62,68,99,70,61,65,82,39,722,5.0,Vidya Mandir


In [ ]:
# Function to get the 10th ranker's marks
def get_10th_rank_marks(df):
    rank_10_df = df[df['Rank'] == 10]
    if not rank_10_df.empty:
        return rank_10_df['Total_Marks'].values[0]
    else:
        return None

In [ ]:
rank_10_marks = []
for school_name in sheet_names:
    # Load data for the school for the year 2020
    data = load_data_for_year(school_name, '2020')

    # Calculate total marks
    data = calculate_total_marks(data, all_subject_columns)

    # Rank students within the school
    ranked_data = rank_students(data)

    # Get the 10th ranked student's marks
    marks = get_10th_rank_marks(ranked_data)
    if marks is not None:
        rank_10_marks.append({'School': school_name, 'Rank_10_Marks': marks})


In [ ]:
# Convert the list to a DataFrame
rank_10_marks_df = pd.DataFrame(rank_10_marks)

# Sort the DataFrame by Rank_10_Marks in descending order
sorted_rank_10_marks_df = rank_10_marks_df.sort_values(by='Rank_10_Marks', ascending=False)

# Display the result
print(sorted_rank_10_marks_df)

          School  Rank_10_Marks
0   Vidya Mandir            705
3       Birla HS            673
4  International            660
1     St. Joseph            649
2            DPS            614


# **Question 3 : Find out students with the highest improvement for each subject from 2019-21 combining all the schools together**

In [ ]:
# Load and combine data from all schools for each year
def load_and_combine_data(year):
    combined_data = []
    for school_name in sheet_names:
        df = load_data_for_year(school_name, year)
        df['School'] = school_name
        combined_data.append(df)
    combined_df = pd.concat(combined_data, ignore_index=True)
    return combined_df

In [ ]:
data_2019 = load_and_combine_data('2019')
data_2020 = load_and_combine_data('2020')
data_2021 = load_and_combine_data('2021')

In [ ]:
# Ensure that student Rolls are consistent across years
data_2019.set_index('Student Roll', inplace=True)
data_2020.set_index('Student Roll', inplace=True)
data_2021.set_index('Student Roll', inplace=True)

In [ ]:
# Calculate improvement for each student in each subject from 2019 to 2021
improvement = {}
for subject in all_subject_columns:
    improvement[subject] = (data_2021[subject] - data_2019[subject]).reset_index()

In [ ]:
# Find the student with the highest improvement for each subject
highest_improvement = []
for subject, improvement_df in improvement.items():
    max_improvement_row = improvement_df.loc[improvement_df[subject].idxmax()]
    highest_improvement.append({
        'Student Roll': max_improvement_row['Student Roll'],
        'Subject': subject,
        'Improvement': max_improvement_row[subject]
    })

# Convert the list to a DataFrame
highest_improvement_df = pd.DataFrame(highest_improvement)

# Display the result
print(highest_improvement_df)

    Student Roll             Subject  Improvement
0           1013             Physics           63
1           1017    Computer Science           82
2           3008         Mathematics           67
3           2004             History           51
4           1011               Hindi           71
5           1010           Geography           61
6           4019             Biology           54
7           3005             English           59
8           1008  Physical Education           58
9           1018              Civics           65
10          3020           Chemistry           65


# **Question 4: Identify best school for Arts, Science and Commerce streams based on marks scored by students in respective subjects for those streams in last three years**

In [ ]:
def load_and_combine_data(year):
    combined_data = []
    for school_name in sheet_names:
        df = load_data_for_year(school_name, year)
        df['School'] = school_name
        combined_data.append(df)
    combined_df = pd.concat(combined_data, ignore_index=True)
    return combined_df

In [ ]:
# Load data for all three years
data_2019 = load_and_combine_data('2019')
data_2020 = load_and_combine_data('2020')
data_2021 = load_and_combine_data('2021')

# Combine data for all three years
data_all_years = pd.concat([data_2019, data_2020, data_2021])

In [ ]:
# Calculate average marks for each school and stream
def calculate_average_marks(data, subject_columns):
    average_marks = data.groupby('School')[subject_columns].mean().mean(axis=1)
    return average_marks

# Calculate average marks for each stream
arts_avg_marks = calculate_average_marks(data_all_years, subject_columns['Arts'])
science_avg_marks = calculate_average_marks(data_all_years, subject_columns['Science'])
commerce_avg_marks = calculate_average_marks(data_all_years, subject_columns['Commerce'])


In [ ]:
# Identify the best school for each stream
best_school_arts = arts_avg_marks.idxmax()
best_school_science = science_avg_marks.idxmax()
best_school_commerce = commerce_avg_marks.idxmax()

In [ ]:
print(f"Best school for Arts stream: {best_school_arts}")
print(f"Best school for Science stream: {best_school_science}")
print(f"Best school for Commerce stream: {best_school_commerce}")

Best school for Arts stream: Birla HS
Best school for Science stream: International
Best school for Commerce stream: Vidya Mandir


# **Question 5: If the marks obtained for each subject can be categorised under 5 sections like below:**
    
          <=20 - Very Poor
    
    >20 - <=40 - Poor
    
    >40 - <=60 - Average
    
    >60 - <=80 - Good
    
    >80 - <=100 - Very Good
    

**Then calculate for each school how many students were in each category based on the avg. marks obtained each year**

In [ ]:
# Calculate average marks for each student across all subjects for each year
data_all_years['Average_Marks'] = data_all_years[all_subject_columns].mean(axis=1)

In [ ]:
# Define the categorization function
def categorize_marks(avg_marks):
    if avg_marks <= 20:
        return 'Very Poor'
    elif avg_marks <= 40:
        return 'Poor'
    elif avg_marks <= 60:
        return 'Average'
    elif avg_marks <= 80:
        return 'Good'
    else:
        return 'Very Good'

In [ ]:
# Apply the categorization function to the average marks
data_all_years['Category'] = data_all_years['Average_Marks'].apply(categorize_marks)

In [ ]:
# Count the number of students in each category for each school
category_counts = data_all_years.groupby(['School', 'Category']).size().unstack(fill_value=0)

In [ ]:
print(category_counts)

Category       Average  Good
School                      
Birla HS            20    40
DPS                 41    19
International       25    35
St. Joseph          43    17
Vidya Mandir        22    38


# Question 6: Which is the best school for each year 2019, 2020 and 2021 based on highest no. of students in Good and Very Good category?

In [ ]:
import pandas as pd

# Load Excel files (assuming files are uploaded to Google Colab)
excel_files = {
    '2019': pd.ExcelFile('/content/Bangalore Schools 2019.xlsx'),
    '2020': pd.ExcelFile('/content/Bangalore Schools 2020.xlsx'),
    '2021': pd.ExcelFile('/content/Bangalore Schools 2021.xlsx')
}

# Get sheet names (i.e., school names) from one of the files (assuming all files have the same sheets)
sheet_names = excel_files['2020'].sheet_names

# Function to load data for a specific year and school
def load_data_for_year(school_name, year):
    df = excel_files[year].parse(school_name)
    return df

# Define the list of subject columns for each stream
subject_columns = {
    'Arts': ['Hindi', 'English', 'History', 'Geography', 'Civics'],
    'Science': ['Mathematics', 'Physics', 'Chemistry', 'Biology', 'Computer Science'],
    'Commerce': ['Hindi', 'English', 'Mathematics', 'Computer Science', 'Physical Education']
}

# Merge all subject columns into one list
all_subject_columns = list(set([subject for subjects in subject_columns.values() for subject in subjects]))

# Function to load and combine data from all schools for each year
def load_and_combine_data(year):
    combined_data = []
    for school_name in sheet_names:
        df = load_data_for_year(school_name, year)
        df['School'] = school_name
        combined_data.append(df)
    combined_df = pd.concat(combined_data, ignore_index=True)
    combined_df['Year'] = year
    return combined_df

# Load data for all three years
data_2019 = load_and_combine_data('2019')
data_2020 = load_and_combine_data('2020')
data_2021 = load_and_combine_data('2021')

# Combine data for all three years
data_all_years = pd.concat([data_2019, data_2020, data_2021])

# Calculate average marks for each student across all subjects for each year
data_all_years['Average_Marks'] = data_all_years[all_subject_columns].mean(axis=1)

# Define the categorization function
def categorize_marks(avg_marks):
    if avg_marks <= 20:
        return 'Very Poor'
    elif avg_marks <= 40:
        return 'Poor'
    elif avg_marks <= 60:
        return 'Average'
    elif avg_marks <= 80:
        return 'Good'
    else:
        return 'Very Good'

# Apply the categorization function to the average marks
data_all_years['Category'] = data_all_years['Average_Marks'].apply(categorize_marks)


In [ ]:
# Filter data for "Good" and "Very Good" categories
filtered_data = data_all_years[data_all_years['Category'].isin(['Good', 'Very Good'])]

# Count the number of students in "Good" and "Very Good" categories for each school for each year
category_counts = filtered_data.groupby(['Year', 'School']).size().reset_index(name='Count')

# Identify the best school for each year
best_schools = category_counts.loc[category_counts.groupby('Year')['Count'].idxmax()]

# Display the best schools for each year
print(best_schools)

    Year        School  Count
0   2019      Birla HS     15
5   2020      Birla HS     14
14  2021  Vidya Mandir     13


# Question 7 : Which is the fastest-growing School in Bangalore (Overall and Streamwise)?

In [ ]:
# Function to load data for a specific year and school
def load_data_for_year(school_name, year):
    df = excel_files[year].parse(school_name)
    return df

# Define the list of subject columns for each stream
subject_columns = {
    'Arts': ['Hindi', 'English', 'History', 'Geography', 'Civics'],
    'Science': ['Mathematics', 'Physics', 'Chemistry', 'Biology', 'Computer Science'],
    'Commerce': ['Hindi', 'English', 'Mathematics', 'Computer Science', 'Physical Education']
}

# Merge all subject columns into one list
all_subject_columns = list(set([subject for subjects in subject_columns.values() for subject in subjects]))

# Function to load and combine data from all schools for each year
def load_and_combine_data(year):
    combined_data = []
    for school_name in sheet_names:
        df = load_data_for_year(school_name, year)
        df['School'] = school_name
        combined_data.append(df)
    combined_df = pd.concat(combined_data, ignore_index=True)
    combined_df['Year'] = year
    return combined_df

# Load data for all three years
data_2019 = load_and_combine_data('2019')
data_2020 = load_and_combine_data('2020')
data_2021 = load_and_combine_data('2021')

# Combine data for all three years
data_all_years = pd.concat([data_2019, data_2020, data_2021])

# Calculate average marks for each student across all subjects for each year
data_all_years['Average_Marks'] = data_all_years[all_subject_columns].mean(axis=1)

# Function to calculate average marks for each school and stream
def calculate_average_marks(data, subject_columns):
    average_marks = data.groupby(['School', 'Year'])[subject_columns].mean().mean(axis=1).reset_index(name='Average_Marks')
    return average_marks


In [ ]:
# Calculate average marks for each stream
arts_avg_marks = calculate_average_marks(data_all_years, subject_columns['Arts'])
science_avg_marks = calculate_average_marks(data_all_years, subject_columns['Science'])
commerce_avg_marks = calculate_average_marks(data_all_years, subject_columns['Commerce'])

# Calculate overall average marks for each school
overall_avg_marks = calculate_average_marks(data_all_years, all_subject_columns)

# Function to calculate growth rate
def calculate_growth_rate(avg_marks):
    growth = avg_marks.pivot(index='School', columns='Year', values='Average_Marks')
    growth['Growth_Rate'] = growth['2021'] - growth['2019']
    growth = growth.reset_index()
    return growth[['School', 'Growth_Rate']]

# Calculate growth rates for each stream
arts_growth_rate = calculate_growth_rate(arts_avg_marks)
science_growth_rate = calculate_growth_rate(science_avg_marks)
commerce_growth_rate = calculate_growth_rate(commerce_avg_marks)


In [ ]:
# Calculate overall growth rate
overall_growth_rate = calculate_growth_rate(overall_avg_marks)

# Identify the fastest-growing school for each stream and overall
fastest_growing_arts_school = arts_growth_rate.loc[arts_growth_rate['Growth_Rate'].idxmax()]
fastest_growing_science_school = science_growth_rate.loc[science_growth_rate['Growth_Rate'].idxmax()]
fastest_growing_commerce_school = commerce_growth_rate.loc[commerce_growth_rate['Growth_Rate'].idxmax()]
fastest_growing_overall_school = overall_growth_rate.loc[overall_growth_rate['Growth_Rate'].idxmax()]

In [ ]:
# Display the fastest-growing schools
print("Fastest-growing school for Arts stream:")
print(fastest_growing_arts_school)
print("\nFastest-growing school for Science stream:")
print(fastest_growing_science_school)
print("\nFastest-growing school for Commerce stream:")
print(fastest_growing_commerce_school)
print("\nFastest-growing school overall:")
print(fastest_growing_overall_school)

Fastest-growing school for Arts stream:
Year
School         Vidya Mandir
Growth_Rate            2.59
Name: 4, dtype: object

Fastest-growing school for Science stream:
Year
School         Vidya Mandir
Growth_Rate            -2.4
Name: 4, dtype: object

Fastest-growing school for Commerce stream:
Year
School         Vidya Mandir
Growth_Rate            8.52
Name: 4, dtype: object

Fastest-growing school overall:
Year
School         Vidya Mandir
Growth_Rate        1.245455
Name: 4, dtype: object
